In [174]:
import matplotlib.animation as animation
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
from ipywidgets import interact
from scipy.optimize import curve_fit

ModuleNotFoundError: No module named 'scipy._lib'

In [225]:
def lorentz(X, mean, sigma, scale, offset):
    return scale/(np.pi * sigma * (1 + ((X - mean)/sigma)**2)) + offset

def lorentz_shift(X, mean, sigma, scale, offset):
    return -(2 * sigma * (-mean + X)) / (np.pi * (mean**2 + sigma**2 - 2*mean*X + np.power(X, 2))**2)*scale + offset

def both_lorentz(X, mean1, sigma1, scale1, offset1, mean2, sigma2, scale2, offset2):
    return lorentz(X, mean1, sigma1, scale1, offset1) + lorentz_shift(X, mean2, sigma2, scale2, offset2)

class Map:
    def __init__(self, src, x, y):
        raw_data = np.loadtxt(src, delimiter='\t')
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(raw_data[2:], columns = ['RamanShift'] + list(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T

        self.cov_matrix = np.cov(self.data_matrix)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        
        self.base_vectors = np.dot(self.vectors, self.data_matrix)
    
    def reconstruct(self, count):
        return np.dot(self.vectors[:count].T, self.base_vectors[:count])
    
    def plotComponent(self, id):
        fig = px.imshow(np.reshape(self.vectors[id], self.shape), title = 'Komponent ' + str(id))
        return fig
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self):
        fig = px.bar(self.weights, title = 'Wagi')
        return fig
    
    def plotSpectrum(self, num):
        fig = px.line(self.data, x = 'RamanShift', y = num)
        return fig
    
    def plotSpectrums(self, ids):
        fig = go.Figure()
        for num in ids:
            fig.add_scatter(x = self.raman_shift, y = self.data_matrix[num], mode='lines', name = 'ID: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
    
    def plotBaseVectors(self, count):
        fig = go.Figure()
        for num in range(count):
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[num], mode='lines', name = 'Num: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
            
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: np.linalg.norm([x, y] - e), self.map)))
    
    def posToId(self, x, y):
        return self.shape[0] * y + x
    
    def draw_curve(self, curve, params, a, b, **draw):
        fig = go.Figure()
        if 'spectrum' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.data_matrix[draw['spectrum']], mode='lines')
        if 'base_vector' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[draw['base_vector']], mode='lines')
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        fig.add_scatter(mode='lines', x = data_slice, y = curve(data_slice, *params))
        return fig
        
    
map_a = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt', 34, 34)

In [176]:
# map_a.plotCovMatrix().show()
map_a.plotWeights().show()

In [134]:
map_a.plotSpectrums([0, 10, 78, 150, 450, 765]).show()

In [155]:
map_a.plotBaseVectors(5)

In [177]:
r = map_a.reconstruct(5)
d = map_a.data_matrix - r
i = 78
f = map_a.plotSpectrum(i)
f.add_scatter(x = map_a.raman_shift, y = r[i], mode='lines', name = 'Rekonstrukcja')
f.add_scatter(x = map_a.raman_shift, y = d[i], mode='lines', name = 'Roznica')
f.update_layout(hovermode="x")
f.show()

In [156]:
i = 1
px.line(x = map_a.raman_shift, y = map_a.base_vectors[i]).show()
map_a.plotComponent(i).show()

In [224]:
d = map_a.data_matrix - map_a.reconstruct(5)
img = np.reshape(np.sum(np.abs(d), axis = 1), (34, 34))
px.imshow(img)

In [226]:
map_a.plotSpectrum(map_a.posToId(6, 15))

In [212]:
# peak1 = (1380, 1410)
# peak2 = (730, 770)
# peak3 = (1300, 1380)

# map_a.draw_curve(lorentz, (1392, 2, -1.45 * 10**6, -7000), 1330, 1450, base_vector = 0)
map_a.draw_curve(lorentz, (750, 5, -1.3 * 10**5, -5000), 730, 770, base_vector = 0)

In [218]:
map_a.draw_curve(lorentz, (750, 5.5, 1.3 * 10**4, 1800), 730, 770, base_vector = 1)

In [232]:
map_a.draw_curve(lorentz, (750, 5.5, -2.2 * 10**4, -600), 730, 770, base_vector = 2)


In [237]:
map_a.draw_curve(lorentz, (750, 5.5, -9.5 * 10**4, -4000), 730, 770, base_vector = 3)


In [246]:
map_a.draw_curve(lorentz, (750, 5.5, 2.5 * 10**4, -2300), 730, 770, base_vector = 4)
